In [29]:
'''Part of code contributed by Chris'''
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
%matplotlib inline

file_path = '/Users/haowei/Documents/gal/Day37_case_study2/case_study2/Churn/churn_train.csv'
def import_data(file_path):
    df = pd.read_csv(file_path)
    return df

def add_churn(df):
    df['last_trip_date'] = pd.to_datetime(df['last_trip_date'])
    condition = df['last_trip_date'] < '2014-06-01'
    df['churn'] = 1
    df.ix[~condition, 'churn'] = 0
    return df
df = import_data(file_path)
df = add_churn(df)
df.head(10)

def add_dummies(df, target_col):
    d = pd.get_dummies(df[target_col])
    df_list = [df,d]
    df = pd.concat(df_list,axis =1)
    df = df.drop(target_col, axis =1)
    return df
df_c = add_dummies(df, 'city')
df_c = add_dummies(df_c,'phone')
df_c.head()

df_drop = df[~df['avg_rating_of_driver'].isnull()]
r = search_rate(df_drop, 0.7, 'avg_rating_of_driver' )
print r

0.590672801147
5.0
0.590672801147
4.0
0.646903205727
4.7
0.539343292412
3.0
0.728550295858
3.0


In [ ]:
def crossVal(x_train, y_train, model, fold =5 ):
    #x_train, y_train should be numpy arrary
    # modle is a test modle, e.g., #sklearn.linear_model.LinearRegression()
    kf = sklearn.cross_validation.KFold(len(x_train), fold) 
    mses = []
    for train_ind, test_ind in kf:
        train_x = x_train[train_ind]
        train_y = y_train[train_ind]
        test_x = x_train[test_ind] # for the current q, we only use x_train for KFold
        test_y = y_train[test_ind]
        
        md = model.fit(train_x, train_y)#sklearn.linear_model.LinearRegression().fit(train_x, train_y)
        y_hat = md.predict(test_x)
        mse = sklearn.metrics.mean_squared_error(test_y, y_hat)
        mses.append(mse)
    return np.mean(mses)

df['city'].unique()
df_city = pd.get_dummies(df['city'])
df_city.head()
df_phone = pd.get_dummies(df['phone'])
df_phone.head()
df_with_dummies = df
df_list = [df_phone,df_city,df]
df = pd.concat(df_list,axis =1)

churn_rate = df[df['churn']==1].shape[0]/float(df.shape[0])

df0 = df[df['churn'] ==1]
df1 = df[df['avg_rating_of_driver'].isnull()]
df2 = df1[df1.churn == 1]
print 'churned ',df0.shape[0]
print 'churn rate', churn_rate
print 'missing avg_rating_of_driver',df1.shape[0]
print 'missing avg_rating_of_driver and churned', df2.shape[0]
print 'churn rate for missing avg_rating_of_driver', df2.shape[0]/float(df1.shape[0])

df3 = df[df['avg_rating_of_driver'] < 0.5*df['avg_rating_of_driver'].mean()]
df4 = df3[df3.churn == 1]

df5 = df[df['avg_rating_of_driver'] >= df['avg_rating_of_driver'].mean()]
df6 = df5[df5.churn == 1]

df7 = df[df['avg_rating_by_driver'].isnull()]
df8 = df7[df7.churn == 1]

# print df3.shape[0]
# print df4.shape[0]
# print df4.shape[0]/float(df3.shape[0])

# print df5.shape[0]
# print df6.shape[0]
# print df6.shape[0]/float(df5.shape[0])

print 'missing avg_rating_by_driver',df7.shape[0]
print 'missing avg_rating_by_driver and churned',df8.shape[0]
print 'churn rate for missing avg_rating_by_driver',df8.shape[0]/float(df7.shape[0])
df.describe()

In [37]:
# plan A: creat new cateroty column rating_cat where NaN rating and 
#ratings<0.5*mean will be assigned to 0, ratings>0.5*mean will be 1

def add_rating_cat(df):
    condition1 = df['avg_rating_of_driver'].isnull() 
    condition2 = df['avg_rating_of_driver'] < 0.5*df['avg_rating_of_driver'].mean()
    df['rating_category'] = 1
    df.ix[condition1|condition2, 'rating_category'] = 0
    df.drop('avg_rating_of_driver', axis = 1, inplace =True)
    return df
# newdf = add_rating_cat(fullData)
# newdf.head()

def impute_rating(df):
    condition = df['avg_rating_of_driver'].isnull() 
    df.ix[condition, 'avg_rating_of_driver'] = 0.5*df['avg_rating_of_driver'].mean()
    return df
df_new1 = impute_rating(df)
df_new1.head(100)
list_1 = ['churn','weekday_pct','avg_dist','avg_rating_by_driver','avg_rating_of_driver','avg_surge','surge_pct','trips_in_first_30_days']
list_2 = ['weekday_pct','avg_dist','surge_pct','trips_in_first_30_days','avg_surge']
list_3 = ['churn','avg_dist','avg_rating_by_driver','avg_rating_of_driver']

df_plot = df[list_3]
# ax = scatter_matrix(df_plot, diagonal = 'kde', figsize = (8,8), alpha = 0.4)
# plt.show()

x1 = df_new1['avg_rating_by_driver']
y1 = df_new1['avg_rating_of_driver']
c1 = np.array(df_new1['churn'])
print len(x1), len(y1), len(c1)
# plt.scatter(x1,y1, c = c1, alpha = 0.5)
# plt.ylabel('avg_rating_of_driver')
# plt.xlabel('avg_rating_of_driver')

churn_rate = df[df['churn']==1].shape[0]/float(df.shape[0])

def search_rate(df, target_rate, target_col):
    churn_rate = df[df['churn']==1].shape[0]/float(df.shape[0])
    print churn_rate
    values = sorted(df[target_col].unique(), reverse = True)
    #values = df[target_col].unique()
    for item in values:
        print item
        dft = df[df[target_col] <= item]
        dft_churn = dft[dft['churn'] ==1].shape[0]/float(dft.shape[0])
        print dft_churn
        if dft_churn >=target_rate:
            return item
    else:
        return df[target_col].min()

r = search_rate(df, 0.79, 'avg_rating_of_driver' )
print r
    

40000 40000 40000
0.6242
5.0
0.6242
4.9
0.575886948726
4.8
0.598781707428
4.7
0.622942643392
4.6
0.638732698962
4.5
0.65491355778
4.4
0.666325027224
4.3
0.680251622833
4.2
0.698944302876
4.1
0.711570372605
4.0
0.722093561834
3.9
0.726684073107
3.8
0.735843020156
3.7
0.747037324178
3.6
0.75480989464
3.5
0.761494587359
3.4
0.76935267585
3.3
0.773466371027
3.2
0.779482021624
3.1
0.781960586155
3.0
0.784517766497
2.9
0.789860189239
2.8
0.790776630358
2.8
